In [1]:
from collections import Counter
from itertools import count
import math

import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import numpy as np
import pandas as pd
from scipy.sparse import lil_matrix, csr_matrix
from tqdm import tqdm, tqdm_notebook

%matplotlib inline
stopwords = set(stopwords.words('english'))

# Cosine similarity

First we read in the data and process the abstracts by removing `NaN`s, converting to lowercase, and removing stopwords.

In [2]:
df = pd.read_csv("../data/full_arxiv_cs_clean.csv", index_col="Unnamed: 0", low_memory=False)
# selected = df[~df.abstract.isnull()][['abstract', 'id']]
clean_abstracts = df[~df.abstract.isnull()].abstract.str.lower()
clean_abstracts = clean_abstracts.str.replace(r"[\[\].()$,:;\"%{}\-\\/<>+=_~'^]", " ")
clean_abstracts = clean_abstracts.str.replace(r" {2,}", " ")
clean_abstracts = clean_abstracts.apply(lambda x: [i for i in x.strip().split()
                                                   if i not in stopwords])

Next, we can generate word counts for each document with a Python `Counter` like so.

In [3]:
clean_abstracts = clean_abstracts.apply(lambda x: Counter(x))

To get the full list of words in all documents, we can go through each `Counter` and add the keys to a `set` of words.

In [4]:
unique_words = set()

def add_words(s, doc):
    [s.add(i) for i in doc.keys()]
    
clean_abstracts.apply(lambda x: add_words(unique_words, x));
unique_words = dict(zip(unique_words, count()))

In [5]:
def map_word_ids(row):
    d = dict()
    for k, v in row.items():
        d.update({unique_words[k]: row[k]})
    return d

clean_abstracts = clean_abstracts.apply(map_word_ids)

In [17]:
inv_unique_words = {v:k for k,v in unique_words.items()}

In [18]:
import json

with open("../data/unique_words.json", "w") as f:
    json.dump(unique_words, f, indent=True)

with open("../data/unique_words_inv.json", "w") as f:
    json.dump(inv_unique_words, f, indent=True)

In [76]:
# sp_counts = lil_matrix((clean_abstracts.shape[0], len(unique_words)), dtype=np.uint8)

# for ix, row in tqdm(enumerate(clean_abstracts), total=clean_abstracts.shape[0]):
#     for k, v in sorted(row.items(), key=lambda x: x[0]):
#         sp_counts[ix, k] = v

# sp_counts = sp_counts.tocsr()

In [107]:
# import pickle

# with open("../data/cs_sparse_matrix.pickle", "wb") as f:
#     pickle.dump(sp_counts, f)
    
# with open("../data/cs_unique_words_map.pickle", "wb") as f:
#     pickle.dump(unique_words, f)

# with open("../data/ca_counts.pickle", "wb") as f:
#     pickle.dump(clean_abstracts, f)

In [310]:
def dot(a, b):
    """Takes two sparse vectors and finds their dot products.
    
    Parameters
    ----------
    a, b -- Sparse vectors represented as Python Counters
    
    Returns
    -------
    c -- Dot product of `a` and `b`
    """
    c = 0
    for k in a.keys() & b.keys():
        c += a[k] * b[k]
    return c

def norm(a):
    """Finds the L2 norm of a sparse vector.
    
    Parameters
    ----------
    a -- Sparse vector represented as a Python Counter
    
    Returns
    -------
    b -- Norm of `a`
    """
    return np.linalg.norm(np.fromiter(a.values(), dtype=int), 2)

def cosine_similarity(doc1, doc2):
    """Finds the cosine similarity between two documents.
    
    Parameters
    ----------
    doc1, doc2 -- Sparse vectors of word counts represented as Python Counters
    
    Returns
    -------
    Cosine similarity between `doc1` and `doc2`
    """
    return dot(doc1, doc2)/(norm(doc1)*norm(doc2))

def cosine_distance(doc1, doc2):
    """Finds the cosine distance between two documents.
    
    Parameters
    ----------
    doc1, doc2 -- Sparse vectors of word counts represented as Python Counters
    
    Returns
    -------
    Cosine distance between `doc1` and `doc2`
    """
    return 1-cosine_similarity(doc1, doc2)

In [319]:
def most_similar(doc, n=100):
    """Returns the n most similar documents.
    
    Parameters
    ----------
    doc -- Document of interest
    n -- Number of results to return (optional)
    
    Returns
    -------
    similarities -- A size-n vector of document similarities, ignoring the same document"""
    similarities = clean_abstracts.apply(lambda x: cosine_similarity(doc, x))
    return similarities.sort_values(ascending=False)[1:(1+n)]

In [26]:
[{v: k} for k,v in unique_words.items()][134007]

KeyboardInterrupt: 

In [15]:
clean_abstracts.apply(lambda x: [{k:v} for k,v in x.items()]).to_json("../data/clean_abstracts.json")

In [17]:
clean_abstracts.to_json("../data/clean_abstracts.json", orient="index")

In [13]:
df.id.to_csv("../data/arxivids.csv")

In [333]:
df.loc[most_similar(clean_abstracts[0], n=100).index, ['abstract','title','authors']]

,abstract,title,authors
164462,"A hypergraph $G=(V,E)$ is $(k,\ell)$-sparse if...",Sparse Hypergraphs and Pebble Game Algorithms,"['Streinu Ileana', 'Theran Louis']"
164446,"A multi-graph $G$ on $n$ vertices is $(k,\ell)...",Pebble Game Algorithms and Sparse Graphs,"['Lee Audrey', 'Streinu Ileana']"
99016,Strictly Chordality-k graphs (SC_k graphs) are...,On strictly Chordality-k graphs,"['Dhanalakshmi S.', 'Sadagopan N.']"
127640,"For two positive integers $k$ and $\ell$, a $(...",On the complexity of finding internally vertex...,"['Araújo Júlio', 'Campos Victor A.', 'Maia Ana..."
57040,We introduce a new notion of resilience for co...,On Coloring Resilient Graphs,"['Kun Jeremy', 'Reyzin Lev']"
93260,In this paper we introduce k-laminar graphs a ...,Read networks and k-laminar graphs,"['Völkel Finn', 'bapteste Eric', 'Habib Michel..."
131754,We give the first polynomial-time algorithms o...,Polynomial-time algorithms for the Longest Ind...,"['Jaffke Lars', 'Kwon O-joung', 'Telle Jan Arne']"
31377,The existential k-pebble game characterizes th...,Lower Bounds for Existential Pebble Games and ...,['Berkholz Christoph:RWTH Aachen University']
20661,Many algorithms have been developed for NP-har...,Approximate Tree Decompositions of Planar Grap...,"['Kammer Frank', 'Tholey Torsten']"
5120,"In this paper, we revisit the split decomposit...",Split decomposition and graph-labelled trees: ...,"['Gioan Emeric', 'Paul Christophe']"


In [322]:
sys.getsizeof(clean_abstracts)/1e6

624.641688

In [49]:
## Running this on the full data set would be ~160 Gb

# similarities = dict()

# def get_similarities(row):
#     id_ = row['index']
#     if id_ % 100 == 0:
#         print(id_)
#     similarities.update({id_: dict()})
#     ca_counts[id_:1000].reset_index().apply(
#         lambda doc: similarities[id_].update(
#             {doc['index']: cosine_similarity(row.abstract, doc.abstract)}
#         ), axis=1)
        
# ca_counts[:1000].reset_index().apply(get_similarities, axis=1);

In [158]:
sp_counts_T = sp_counts.transpose().tocsc()

In [176]:
from numba import jit

In [279]:
def cosine_sim_sp1(ix1, ix2):
    common = np.intersect1d(sp_counts[ix1,:].indices, sp_counts[ix2,:].indices)
    num = sp_counts[ix1,common].dot(sp_counts_T[common,ix2])
    denom = spnorm(sp_counts[ix1,common])*spnorm(sp_counts[ix2,common])
    return (num/denom)[0,0]

def cosine_sim_sp2(ix1, ix2):
    return (sp_counts[ix1,:].dot(sp_counts_T[:,ix2])/spnorm(sp_counts[ix1,:])*spnorm(sp_counts[ix2,:]))[0,0]

@jit
def jitcosine_sim_sp1(ix1, ix2):
    common = np.intersect1d(sp_counts[ix1,:].indices, sp_counts[ix2,:].indices)
    num = sp_counts[ix1,common].dot(sp_counts_T[common,ix2])
    denom = spnorm(sp_counts[ix1,common])*spnorm(sp_counts[ix2,common])
    return (num/denom)[0,0]

@jit
def jitcosine_sim_sp2(ix1, ix2):
    num = sp_counts[ix1,:].dot(sp_counts_T[:,ix2])
    denom = spnorm(sp_counts[ix1,:])*spnorm(sp_counts[ix2,:])
    return (num/denom)[0,0]


def jitcosine_sim_sp3(ix1, ix2):
    common = np.intersect1d(sp_counts[ix1,:].indices, sp_counts[ix2,:].indices, assume_unique=True)
    num = sp_counts[ix1,common].dot(sp_counts_T[common,ix2])
    denom = spnorm(sp_counts[ix1,common])*spnorm(sp_counts[ix2,common])
    return (num/denom)[0,0]


In [283]:
# %timeit cosine_sim_sp1(0, 1)
%timeit cosine_sim_sp2(0, 1)
# %timeit jitcosine_sim_sp1(0, 1)
# %timeit jitcosine_sim_sp2(0, 1)
# %timeit jitcosine_sim_sp3(0, 1)

1.12 ms ± 6.11 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [282]:
def c(i1, i2):
    return cosine_sim_sp2(i1, i2)

%lprun -f cosine_sim_sp2 c(0,1)

Timer unit: 1e-06 s

Total time: 0.00405 s
File: <ipython-input-279-d25862ccd79a>
Function: cosine_sim_sp2 at line 7

Line #      Hits         Time  Per Hit   % Time  Line Contents
     7                                           def cosine_sim_sp2(ix1, ix2):
     8         1       4050.0   4050.0    100.0      return (sp_counts[ix1,:].dot(sp_counts_T[:,ix2])/spnorm(sp_counts[ix1,:])*spnorm(sp_counts[ix2,:]))[0,0]